# Section 2: The building blocks of a Scientific Computing Platform

![Pangeo Logo](images/pangeo_logo.png)

In the first section we introduced some typical users of a scientific compute platform and typical tasks that such users may wish to perform on such a platform. We then looked at 2 models for delivering the compute capability that users require, the desktop model and the cluster model. Each of these has its advantages and disadvantages. The next step is to consider how we could design and build a platform that combines the advantages of different systems and removes (as much as possible) the disadvtages.  In this notebook we will discuss those principles by looking at the key goals of what we'll call the *Pangeo model of Scientific Computing*.

![Pangeo model of scientific computing](images/PangeoModel_diagram.png)

## Goal 1 - An interactive platform that scales

A key tension between the desktop and cluster models of computing that we identified was the tradeoffs between compute which enables an interactive workflow and compute which can operate at scale. The Pangeo model should offer compute with an interactive workflow that can scale to the size of the problem you are working on. The development of cloud computing and associated technologies has gone a long way to making this a realisable goal. Let's consider some of the aspects of what cloud computing has mader easier or possible.

### Affordable data storage

Compute capacity is not very useful without matched data storage capacity which can be accessed quickly, easily and cheaply. Cloud computing providers offer huge storage capacity at affordable prices that located close (physically and topologically) to the compute.

It is important to remember that this storage though is structuted differently to the hard disks in the desktop or cluster which we may be used to. Some important traits of cloud storage
* high latency - responses to requests for data take longer than for a local disk on  desktop as this is networked storage akin to that found on a cluster.
* distributed - the storage and access mechanisms are highlly parallelised for robustness and to meet demand. This means one can usually achieve very high data throughputs.
* The distributed nature of the storage means that you can run into issues around consistency. See [CAP Theorem](https://en.wikipedia.org/wiki/CAP_theorem)

This different structure means to make optimal use of the advtages and minimise the disadvtages of cloud storage, we need to use different formats for our data that are **cloud-optimised**

### Distributed compute

It was mentioned previously that part of what made the desktop model work was *Moore's Law*, which described the rate at which the speed of silicon processors increased. More accurately it actually described the rate at which the desnity of transitors in integrated circuits increased. This mostly translated into increased speed of processing without having to change anything about the code that one had written. Due to hard physical limits, this is no longer true. Instead, the latest chips can do more operations in parallel, and in order to get the best performance out of them our code needs to be execute in parallel as much as possible. 

Luckily, many of the common operations in scientific computing are highly parallelisable and so such operations scale well conceptually, provided we write our code appropriately, as well as the underlying tools being written approriately for parallel execution. 


There are multiple levels of distribution:
* *Task distribution* - completely separate operation that do not depend on one another than can be executed in parallel. This may be on separate piceces of data. This is relatively easy to support and was the main way of distribtuing compute previously. Examples: 
  * Example - Calculate min and max temperature on a global temperature dataset.
* *Data distribution* - individual operations on a dataset, where the operation can be performed separately on different parts of the dataset. This sort of distribution has long been a part of weather models, but has up until recently been less common outside of HPC. Only as datasets have become too large to fit on a single machine has this gained importance. The dataset is split into chunks and calculations on each chunk can be done separately, on separate CPU cores, or nodes or even in competely different data centres (though this is of course inefficient!). 
  * Example - Calculate an annual mean temperature from a global hourly dataset. Mean can be calculated separetly for each chunk of data. 


This distributed nature of compute is another factor in creating **cloud-optimised** datasets, so our data  facilitates our exploitation of this massively distributed computed resource rather than hinders it.

### Scalable compute

With ever increasing volumes of data from a variety of data gathering and observational sources, and even more data being produced from simulation of all kinds of real-world systems and processes, a Pangeo platform has to be able to operate at whatever scale is required of it. However much data we have to process, our system should be able to process it. The advantage of a cluster is that we pool compute resources so individual users and tasks can get access to a larger compute resources for a time. With cloud computing, the scale of resources being pooled is even more substantial, enabling such greater scaling.



### Elastic compute

One real barrier to scaling has been not just getting access to compute of sufficient scale, but how to manage the demand for compute that is not consistent. One may need hundreds of CPU for a short time to process a large dataset, but then not need it for the other 23 hours of the day. In Scientific Computing we call this a **burst** workflow. 

Procuring expensive capacity to be used for only a small portion of each day is not cost effective. As a result, when procuring compute infrastructure, the compute required is averaged out over 24 hours, or 7 days in a week or 365 days in a year, so we have to wait a long time for a job to finish so we dont waste our investment. This is dealt with to some extent by clusters. different people are using the cluster simultaneously and hopefully have different timings of peek and trough demands. This helps keep the cluster busy. However weekend, holidays and other factors tend to reduce the effectiveness of this model. What happens is therefore is we adapt our scientific analysis to fit how the compute resource works, rather than the other way around. 

What we want to deal with the *bursting* nature of the work is to have **elastic** compute capacity. We can provision the compute we need for a job, then relinquish it when we are finished. Most importantly, we only want to pay for the time we are using it. This transforms our approach to capacity. In this model it costs the same to use 1 CPU for 1000 minutes as to use 1000 CPUs for 1 minute.  


### Interactive workflow

The ability to quickly provision, and subsequently relinquish, huge amounts of compute when we need it and only pay for the time we need it for, means we can meet both requirements of our system. Our system can be interactive, and when it needs to perform a big operation, it has the compute it needs available. This system is definitely not going to be a simple desktop, so how is the interactivity going to work? There are multiple approaches, all of which follow the distributed principle we mentioned earlier, with the interactivity able to be served on a different compute instance to the heavy lifting required for particular operations. This could be:
* interaction on a desktop, calling out to the cloud when required. The problem may be then how is the data (or metadata available in both places)
* a remote desktop connection to a VM on the cloud


In both cases a fair amount of technical know how is required to connect the bits together. We don't want our users, particularly those focused on scientific research or applications, to have to know about such things. A better solution that is widely used in the community is to have some sort of interactive compute server where which handles the interactivity and the scaling. 


## Goal 2 - Reproducible Research

Computing underlies in some way many of the scientific results published in peer-reviewed jounral, through statistical analysis, visulaisation of data, simulations etc. The source code that produces the published results is thus a key output of the scientific process. In order to trust the scientific result derived from the source code and the undelying libraries, we need to trust that the code is doing what it set out to do (verification) and that what the software aims to do is scientitfically correct (validation) 
[More info](https://en.wikipedia.org/wiki/Software_verification_and_validation#:~:text=In%20software%20project%20management%2C%20software,to%20as%20software%20quality%20control.)


One way of building trust in that software is that others hsould be able to access the code and data, and using the same code and data as used for the research should be able to reproduce the published results. One big barrier to reproducible research of this soft has been that the source code created for a project is only a small part of what is required to reproducible results. Even a fairly small piece of sioftware will have a large set of libraries that it depends on, and using a slightly different version of dependency can change the results, not because there is anything wrong withn the software written by the scientist. Reproducing the environment in which the research software was run reliably and easily is thus a crucial part of reporducible research. Modern environment management tools, such `pip` and `conda`, or lightweight virtualisation technologies such as *containerisation* technologies, e.g. docker, make this a lot easier. 

This sort of environemnt management to facilitate reproducible research is a crucial part of a a Pangeo instance.

## Goal 3 - Shareable research

The goal of shareable research is closely aligned with the reproducible research and is made possible by many of the same components. Historically sharing of research has been done just through what is published in the peer-reviewed journal. The element underpinning those results have not been shared by default. Today we recognise that the elements that underpin the published results, including source code and environments used to generate the results, are also very important to other researchers and if we want to share our results, we want interested parties to see all elements of that research. We need to be able to share our code and data. Other researchers can pick apart our research pipeline and interrogate each part themselves. This is partly to review research results, but also so elements that are useful can be pulled and reused in other research that shares common elements. A common, easily accessible platform for computing makes it much easier to share research pipelines with others

## Goal 4 - Cost effective compute

In many ways the barriers to a scalable, interactiver platform suitable for today's scientific computing come down to one concept: cost. We could, to a certain extent, create such a platform by spending enough money to give each scientist a large enough desktop, though in many domains the size of data to be processed makes even this impractical. Cost limits are usually hit first. Our Pangeo platform needs to provide the interactive compute which scales elastically to meet demand at a cost that is affordadble for your average researcher/research institution. Cloud computing service have made this possible. One pays only for the resources that are actually used while running your job. When the job is finished, the resource is relinquished and no further cost is incurred (for compute, any data stored needs to continue to be stored usually, which incurs an ongoing cost, but this is relatively small).

## Goal 5 - Separation of concerns
A lot of what has been described so far is possible through other computing models, but it requires a lot of knowledge of the technical details of the system to set up a job that scales to the data being processed. In general, one has to manually determine what resources will be needed for the job you wish to process, rather than the platform determining from the job what is required based on the submitted request. This has 2 costs. The first is that scientists need to spend a lot of time familiarising themselves with the details of how to scale up their compute jobs, time which could be better spent on their core research questions rather than technical details. The second problem is that scientific compute job, and also our scientific inquiry, is shaped around the specific implementation of the platform we are using, to make running our computing tasks easier. This limits the sort of questions we ask in our research.

Instead, we want to separate our scientific research from our computing infrastructure, through a separation of concerns. Software Engineers will focus on the best way to use the infrastructure to support interactive compute at scale. They will provide tools for scientists who will not need to think about the details of the computing infrastructure, but can instead focus on their core research and scientific questions they want to ask.

## Building a real system

![Tools for building Pangeo](images/pangeoTechLogos.png)

So we have some lovely goals and a basic idea of how they would fit together. What can we do next to make a real platform that's achieve's the goals we have set for it? Do we have to start from scratch to build each of the compoments?

Fortunately not! Many different communities have been thinking about and developing software libraries that achieve many of these goals and address many of the our concerns. The Pangeo community has primarily come together around bringing these disparate technologies together to into a recipe for creating such a scientific platform. As we will we see, a particular instance of the Pangeo Model does not have a fixed set of tools, but rather tools that fill the requirement of the various elements we have discussed e.g. a way of scaling compute elastically, a means of interacting with data, a cloud computing service etc. Different instances of the Pangeo Model swap different specific tools in and out of their stack for different elements. We will will look at some of the specific tools we could use to build a specific instance of the pPangeo Model in the next section.